# ニューラルネットワーク入門
https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/1_Learning%20PyTorch/1_3_neural_networks_tutorial_jp.ipynb

In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)

        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [4]:
params = list(net.parameters())
print(len(params))
print(params[1].size())

10
torch.Size([6])


In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
out

tensor([[-0.1134,  0.0673,  0.0509,  0.1061,  0.0791,  0.1460, -0.0335, -0.0583,
          0.0190,  0.0258]], grad_fn=<AddmmBackward0>)

In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [7]:
output = net(input)
target = torch.randn(10)  # 適当な正解targetを用意
target = target.view(1, -1)  # 変数targetの形をoutputに合わせるよう変形
criterion = nn.MSELoss()

loss = criterion(output, target)
loss

tensor(0.8488, grad_fn=<MseLossBackward0>)

In [8]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [9]:
net.zero_grad()

print(net.conv1.bias.grad)

loss.backward()

print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])
tensor([ 0.0011,  0.0023,  0.0016, -0.0049,  0.0126,  0.0074])


In [10]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)

loss.backward()
optimizer.step()